In [1]:
import pandas as pd
import numpy as np
from numpy.random import uniform
from numpy import hstack
import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')

import featuretools as ft
import featuretools.variable_types as vtypes
from sklearn.model_selection import train_test_split, cross_validate

# mlp for multi-output regression
# from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold, cross_val_score

from keras.models import Sequential
from keras.layers import Dense

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import(
    classification_report, confusion_matrix, accuracy_score, mean_squared_error, mean_absolute_error
)
from sklearn.preprocessing import (
    StandardScaler, LabelBinarizer, FunctionTransformer,PolynomialFeatures, OrdinalEncoder
)
# from sklearn_pandas import DataFrameMapper, CategoricalImputer
from sklearn.pipeline import Pipeline, FeatureUnion

Using TensorFlow backend.


In [2]:
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.multioutput import MultiOutputRegressor,RegressorChain
from sklearn.svm import LinearSVR
# from sklearn.multioutput import RegressorChain
from sklearn_pandas import DataFrameMapper

In [3]:
aa = pd.read_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\AA-NACL.csv',header=0,
                    names=['temp','conductivity','velocity','nacl','aa'],dtype='float32')
df = pd.read_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\aa-nacl-generated\\AA_NACL_NewFeatures_Corr.csv',dtype='float64')
df_test = pd.read_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\aa-nacl-generated\\AA_NACL_NewFeatures_Corr_test.csv',dtype='float64')


In [4]:
aa_shuffled = aa.sample(frac=1)

In [20]:
X_train = aa_shuffled.drop(aa_shuffled.iloc[:,3:5],axis=1)
X_train

,temp,conductivity,velocity
94,25.0,17.850000,1509.900024
83,15.0,37.599998,1505.900024
87,25.0,0.880000,1499.900024
8,0.0,9.540000,1416.900024
86,25.0,0.630000,1499.199951
...,...,...,...
73,15.0,14.270000,1481.500000
62,10.0,24.980000,1452.599976
18,0.0,25.360001,1449.300049
70,15.0,14.530000,1478.699951


In [21]:
y_train = aa_shuffled.iloc[:,3:5]
y_train.head()

,nacl,aa
94,1.0,1.5
83,3.0,3.0
87,0.0,1.5
8,1.0,0.5
86,0.0,1.0


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train,test_size=0.2,random_state=42)

In [23]:
X_train_org = X_train.copy()
X_test_org = X_test.copy()
y_train_org = y_train.copy()
y_test_org = y_test.copy()

#### Let's manually create more features

In [48]:
mapper = DataFrameMapper([
    (['temp'],  [SimpleImputer(strategy='mean')]),
#     (['temp'],  [SimpleImputer(strategy='mean'), OrdinalEncoder()]),
    (['conductivity'],  [SimpleImputer(strategy='mean'), StandardScaler()]),
    (['velocity'],  [SimpleImputer(strategy='mean'), StandardScaler()])
], df_out=True)

In [49]:
'''
This class performs feature engineering on the retrieved posts.  It is designed to be used in a pipeline
It takes no special input parameters beyond X (the dataframe to be transformed) and returns a modified version of X
with some additional, calculated columns.
Columns Created:

'''
class TitanicTransformer(TransformerMixin):
    def transform(self, X, **transform_params):
        X['vel-con'] = X['conductivity']/np.median(X['velocity'])
        return X
    def fit(self, X, y=None, **fit_params):
        return self

In [50]:
pipe1a = Pipeline([
    ('map', mapper)
#     ('feature_gen', TitanicTransformer())
])

In [51]:
pipe1a = pipe1a.fit(X_train)
X_train = pipe1a.transform(X_train)

In [52]:
# X_test =  pipe1a.transform(X_test)

### Get Baseline Classification Accuracy Score on this dataframe before we add any new features

#### Start with DecisionTreeClassifier 
* (parameters previously tuned using GridSearchCV on this dataframe)

In [53]:
X_test =  pipe1a.transform(X_test)

In [54]:
y_test.to_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\aa-nacl-generated\\aa-results\\y_train.csv')

In [55]:
#  from sklearn.metrics import explained_variance_score
# explained_variance_score(y_true, y_pred, multioutput='raw_values')

In [56]:
def run_regression(X_train, y_train,X_test,y_test):
    ESTIMATORS = {
        "K-nn": KNeighborsRegressor(),                          # Accept default parameters
        "Linear regression": LinearRegression(),
        "RandomForestRegressor": RandomForestRegressor(max_depth=4, random_state=2),
        "Decision Tree Regressor":DecisionTreeRegressor(max_depth=5),
        "RegressorChain Regressor":RegressorChain(DecisionTreeRegressor()),
        "MultiOutputRegressor Regressor":MultiOutputRegressor(LinearRegression()),
        "RegressorChain_Linear Regressor":RegressorChain(LinearRegression()),
        "MultiOutput_DecisionTree Regressor":MultiOutputRegressor(DecisionTreeRegressor())
    }
    # 9.1 Create an empty dictionary to collect prediction values
    y_test_predict = dict()
    y_mse = dict()
    for name, estimator in ESTIMATORS.items():     
        estimator.fit(X_train, y_train)                    # fit() with instantiated object
        y_test_predict[name] = estimator.predict(X_test)   # Make predictions and save it in dict under key: name
        y_mse[name] = mean_squared_error(y_test, estimator.predict(X_test))
        print('Predicted: %s' % y_mse[name])
    #     print('Predicted: %s' %  y_test_predict[name])
    return y_test_predict
#     return y_test_predict.items()
    
    #     scoring = ['precision_macro', 'recall_macro']
#     cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# #     n_scores = cross_validate(wrapper, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
#     n_scores = cross_val_score(wrapper, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
#     n_scores = np.absolute(n_scores)
#     # summarize performance
#     print('MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [57]:
y_test_predict = run_regression(X_train, y_train,X_test,y_test)

Predicted: 0.6138857
Predicted: 0.5113367
Predicted: 0.16553788196608943
Predicted: 0.3173864532960985
Predicted: 0.08976190335693823
Predicted: 0.51133674
Predicted: 0.5113367715620559
Predicted: 0.08976190335693823


In [58]:
(print('Predicted K-nn: %s' % y_test_predict['K-nn']),
print('Predicted Linear: %s' % y_test_predict["Linear regression"]),
print('Predicted RandomForestRegressor: %s' % y_test_predict["RandomForestRegressor"]),
print('Predicted Decision Tree: %s' % y_test_predict["Decision Tree Regressor"]),
print('Predicted RegressorChain: %s' % y_test_predict["RegressorChain Regressor"]),
print('Predicted MultiOutputRegressor: %s' % y_test_predict["MultiOutputRegressor Regressor"]),
print('Predicted RegressorChain_Linear Regressor: %s' % y_test_predict['RegressorChain_Linear Regressor']),
print('Predicted MultiOutput_DecisionTree Regressor: %s' % y_test_predict['MultiOutput_DecisionTree Regressor']))

Predicted K-nn: [[0.        1.96     ]
 [3.        1.7      ]
 [3.        1.3399999]
 [3.        1.7      ]
 [3.        1.3399999]
 [0.        1.3      ]
 [3.        1.3399999]
 [0.        1.2      ]
 [0.2       1.04     ]
 [1.        1.86     ]
 [3.        1.3399999]
 [1.        1.7      ]
 [1.        1.6      ]
 [0.        1.86     ]
 [0.2       1.04     ]
 [0.2       1.04     ]
 [0.        1.2      ]
 [0.        1.96     ]
 [3.        1.3399999]
 [1.        1.7      ]
 [3.        1.9      ]]
Predicted Linear: [[ 0.38370955  1.4910475 ]
 [ 3.7792811   1.4480689 ]
 [ 2.6134362   1.786139  ]
 [ 3.7981362   1.4320314 ]
 [ 3.2136192   1.6675904 ]
 [ 0.24442434  1.6114506 ]
 [ 2.545775    1.8281527 ]
 [ 0.15178132  1.1649582 ]
 [-0.43532336  1.6070689 ]
 [ 1.0410751   1.5211238 ]
 [ 3.1305618   1.7027953 ]
 [ 0.915326    1.1748804 ]
 [ 1.1684616   1.6652077 ]
 [-0.1565988   1.667093  ]
 [-0.40375674  1.605646  ]
 [-0.5094749   1.5801505 ]
 [ 0.14359736  1.1111804 ]
 [ 0.3965454   1.528060

(None, None, None, None, None, None, None, None)

##### SAVING ALL GENERATED PREDICTIONS


In [43]:
knn_pred=pd.DataFrame(y_test_predict['K-nn'],columns=['nacl','aa'])
lr_pred=pd.DataFrame(y_test_predict['Linear regression'],columns=['nacl','aa'])
rf_pred=pd.DataFrame(y_test_predict['RandomForestRegressor'],columns=['nacl','aa'])
dt_pred=pd.DataFrame(y_test_predict['Decision Tree Regressor'],columns=['nacl','aa'])
rchain_pred=pd.DataFrame(y_test_predict['RegressorChain Regressor'],columns=['nacl','aa'])
multi_or_pred=pd.DataFrame(y_test_predict['MultiOutputRegressor Regressor'],columns=['nacl','aa'])
rchain_linear_pred=pd.DataFrame(y_test_predict['RegressorChain_Linear Regressor'],columns=['nacl','aa'])
multior_dt_pred=pd.DataFrame(y_test_predict['MultiOutput_DecisionTree Regressor'],columns=['nacl','aa'])

# SAVING ALL GENERATED PREDICTIONS
knn_pred.to_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\aa-nacl-generated\\aa-results\\knn_pred.csv')
lr_pred.to_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\aa-nacl-generated\\aa-results\\lr_pred.csv')
rf_pred.to_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\aa-nacl-generated\\aa-results\\rf_pred.csv')
dt_pred.to_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\aa-nacl-generated\\aa-results\\dt_pred.csv')
rchain_pred.to_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\aa-nacl-generated\\aa-results\\rchain_pred.csv')
multi_or_pred.to_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\aa-nacl-generated\\aa-results\\multi_or_pred.csv')
rchain_linear_pred.to_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\aa-nacl-generated\\aa-results\\rchain_linear_pred.csv')
multior_dt_pred.to_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\aa-nacl-generated\\aa-results\\multior_dt_pred.csv')

## FEATURETOOLS FEATURE EXTRACTION

In [461]:
es = ft.EntitySet(id = 'hydrate')
es.entity_from_dataframe(entity_id = 'train_df', dataframe = X_train_org, index = 'Id')
# entities = {
#     "cond_df" : (X_train, "id"),
#     "vel_df" : (X_train, "id")}

C:\Program Data\envs\Tensorflow\lib\site-packages\featuretools\entityset\entity.py:463: UserWarning: index Id not found in dataframe, creating new integer column
  "integer column".format(index))


Entityset: hydrate
  Entities:
    train_df [Rows: 84, Columns: 4]
  Relationships:
    No relationships

In [462]:
# relationships = [("cond_df", "id", "cond_df", "conductivity"),
#                  ("vel_df", "id", "vel_df", "velocity")]

In [463]:
es["train_df"].variables

[<Variable: Id (dtype = index)>,
 <Variable: temp (dtype = numeric)>,
 <Variable: conductivity (dtype = numeric)>,
 <Variable: velocity (dtype = numeric)>]

In [464]:
es = es.normalize_entity(base_entity_id='train_df', new_entity_id='conductivity', index='conductivity')
es = es.normalize_entity(base_entity_id='train_df', new_entity_id='velocity', index='velocity')

*  tell DFS to ignore the PassengerId column( we don't want to generate any features for it) using the command ignore_variables={'Passengers':['PassengerId']}),

In [466]:
feature_matrix, feature_names = ft.dfs(entityset=es, 
    target_entity = 'train_df', 
    max_depth = 2, 
    verbose = 3, 
    n_jobs = 1)

Built 21 features
Elapsed: 00:01 | Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████


In [467]:
Xf_train=feature_matrix.copy() # save the generated features back into our X_Train
feature_matrix.head().T

Id,0,1,2,3,4
temp,4.000000,4.000000,15.000000,4.000000,4.000000
conductivity,29.090000,0.940000,0.680000,0.100000,10.650000
velocity,1468.400024,1431.599976,1470.400024,1421.900024,1438.900024
conductivity.COUNT(train_df),1.000000,1.000000,1.000000,1.000000,1.000000
conductivity.MAX(train_df.temp),4.000000,4.000000,15.000000,4.000000,4.000000
conductivity.MEAN(train_df.temp),4.000000,4.000000,15.000000,4.000000,4.000000
conductivity.MIN(train_df.temp),4.000000,4.000000,15.000000,4.000000,4.000000
conductivity.MODE(train_df.velocity),1468.400024,1431.599976,1470.400024,1421.900024,1438.900024
conductivity.NUM_UNIQUE(train_df.velocity),1.000000,1.000000,1.000000,1.000000,1.000000
conductivity.SKEW(train_df.temp),NaN,NaN,NaN,NaN,NaN


* Repeat the steps for Encoding our training set

In [469]:
feature_matrix_enc, features_enc = ft.encode_features(feature_matrix, feature_names, include_unknown=False)
feature_matrix_enc.head()

,temp,conductivity = 37.599998474121094,conductivity = 26.290000915527344,conductivity = 25.829999923706055,conductivity = 25.360000610351562,conductivity = 25.18000030517578,conductivity = 9.420000076293945,conductivity = 9.359999656677246,conductivity = 9.329999923706055,conductivity = 9.3100004196167,...,velocity.MODE(train_df.conductivity) = 26.290000915527344,velocity.MODE(train_df.conductivity) = 25.829999923706055,velocity.MODE(train_df.conductivity) = 25.360000610351562,velocity.MODE(train_df.conductivity) = 25.18000030517578,velocity.MODE(train_df.conductivity) = 17.940000534057617,velocity.MODE(train_df.conductivity) = 17.850000381469727,velocity.NUM_UNIQUE(train_df.conductivity),velocity.SKEW(train_df.temp),velocity.STD(train_df.temp),velocity.SUM(train_df.temp)
Id,,,,,,,,,,,,,,,,,,,,,
0,4.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,1.0,NaN,NaN,4.0
1,4.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,1.0,NaN,NaN,4.0
2,15.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,1.0,NaN,NaN,15.0
3,4.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,2.0,0.585583,5.033223,14.0
4,4.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,1.0,NaN,NaN,4.0


In [470]:
Xf_train = feature_matrix_enc.copy()
Xf_train.head()

,temp,conductivity = 37.599998474121094,conductivity = 26.290000915527344,conductivity = 25.829999923706055,conductivity = 25.360000610351562,conductivity = 25.18000030517578,conductivity = 9.420000076293945,conductivity = 9.359999656677246,conductivity = 9.329999923706055,conductivity = 9.3100004196167,...,velocity.MODE(train_df.conductivity) = 26.290000915527344,velocity.MODE(train_df.conductivity) = 25.829999923706055,velocity.MODE(train_df.conductivity) = 25.360000610351562,velocity.MODE(train_df.conductivity) = 25.18000030517578,velocity.MODE(train_df.conductivity) = 17.940000534057617,velocity.MODE(train_df.conductivity) = 17.850000381469727,velocity.NUM_UNIQUE(train_df.conductivity),velocity.SKEW(train_df.temp),velocity.STD(train_df.temp),velocity.SUM(train_df.temp)
Id,,,,,,,,,,,,,,,,,,,,,
0,4.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,1.0,NaN,NaN,4.0
1,4.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,1.0,NaN,NaN,4.0
2,15.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,1.0,NaN,NaN,15.0
3,4.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,2.0,0.585583,5.033223,14.0
4,4.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,1.0,NaN,NaN,4.0


Now create a new entity set for test and repeat the steps for adding the Passengers and PClass entities¶

In [471]:
es_tst = ft.EntitySet(id = 'hydrate')
es_tst.entity_from_dataframe(entity_id = 'train_df', dataframe = X_test_org, index = 'Id')
es_tst = es_tst.normalize_entity(base_entity_id='train_df', new_entity_id='conductivity', index='conductivity')
es_tst = es_tst.normalize_entity(base_entity_id='train_df', new_entity_id='velocity', index='velocity')

C:\Program Data\envs\Tensorflow\lib\site-packages\featuretools\entityset\entity.py:463: UserWarning: index Id not found in dataframe, creating new integer column
  "integer column".format(index))


### Now call calculate_feature_matrix on our test entity set and pass in the list of saved features from training¶

In [473]:
feature_matrix_tst = ft.calculate_feature_matrix(features=features_enc, entityset=es_tst)

In [474]:
Xf_test = feature_matrix_tst.copy()
Xf_test.head(2)

,temp,conductivity = 37.599998474121094,conductivity = 26.290000915527344,conductivity = 25.829999923706055,conductivity = 25.360000610351562,conductivity = 25.18000030517578,conductivity = 9.420000076293945,conductivity = 9.359999656677246,conductivity = 9.329999923706055,conductivity = 9.3100004196167,...,velocity.MODE(train_df.conductivity) = 26.290000915527344,velocity.MODE(train_df.conductivity) = 25.829999923706055,velocity.MODE(train_df.conductivity) = 25.360000610351562,velocity.MODE(train_df.conductivity) = 25.18000030517578,velocity.MODE(train_df.conductivity) = 17.940000534057617,velocity.MODE(train_df.conductivity) = 17.850000381469727,velocity.NUM_UNIQUE(train_df.conductivity),velocity.SKEW(train_df.temp),velocity.STD(train_df.temp),velocity.SUM(train_df.temp)
Id,,,,,,,,,,,,,,,,,,,,,
0,0.0,False,False,True,False,False,False,False,False,False,...,False,True,False,False,False,False,1.0,NaN,NaN,0.0
1,15.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,1.0,NaN,NaN,15.0


In [563]:
# double check for nulls
for index,col in enumerate(drop_col):
#     print(col)
    if drop_col[index] in Xf_trainAdd:
        print(col)
        Xf_trainAdd.drop(col, axis=1, inplace=True)

In [399]:
model = Sequential()
model.add(Dense(4, input_dim=in_dim, activation="relu"))
model.add(Dense(3, activation="relu"))
model.add(Dense(out_dim))
model.compile(loss="mse", optimizer="adam")
# model.compile(loss='mae', optimizer='adam')
# print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
 
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 15        
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 8         
Total params: 43
Trainable params: 43
Non-trainable params: 0
_________________________________________________________________


In [401]:
from tensorflow import keras
keras.utils.plot_model(model, "my_first_model_with_shape_info.png", show_shapes=True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [402]:
model.fit(xtrain, ytrain, epochs=100, batch_size=12, verbose=0)

In [403]:
ypred = model.predict(xtest)
print("y1 MSE:%.4f" % mean_squared_error(ytest[:,0], ypred[:,0])) 
print("y2 MSE:%.4f" % mean_squared_error(ytest[:,1], ypred[:,1]))

y1 MSE:22771.3418
y2 MSE:0.9704
